### Introduction to NLP Fundamentals in TensorFlow

    NLP has the goal of deriving information out of natural language (could be sequences text or speech)

    Another common term for NLP problems is sequence to sequence problems (seq2seq)

In [1]:
# Check for GPU
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1650 (UUID: GPU-890aadbc-dc9a-cfdd-ce0d-dd3bdfa0ac28)


In [2]:
 ## GET helper functions
import sys
sys.path.append(r"C:\Users/wjub/01workspace/TensorFlow Developer Certificate ZTM")
import winsound
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import helper_functions

In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

# Get a text dataset

    The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaste or not disasterrs)

    See the original source here : https://www.kaggle.com/competitions/nlp-getting-started/data

In [4]:
helper_functions.download_file("https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip","nlp_getting_started.zip")

In [5]:
helper_functions.unzip_data("nlp_getting_started.zip")

## Visualizing a text data 

        To visualize our text samples, we first have to read them in, one way to do so would be to use Python :

        So another way to do this is to use pandas.

In [6]:
import pandas as pd
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
train_df.head()
# train_df["text"][0]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training dataframe
# frac : nb of % to shuffle 1=100%
train_df_shuffled = train_df.sample(frac=1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# What does the test dataframe look like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class ?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total sample ?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
#  Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of  samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target:{target}", "(real diaster)" if target > 0 else "(not real diaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target:0 (not real diaster)
Text:
What's the police or ambulance number in Lesotho? Any body know?

---

Target:1 (real diaster)
Text:
@ThomasHCrown My grandfather was set to be in the first groups of Marines to hit Japan in Operation Olympic. 95% casualty rate predictions

---

Target:1 (real diaster)
Text:
Today marks the 70th anniversary of the bombing of Hiroshima a city I visited and was humbled by in November 2013 http://t.co/AcC1z5Q9Zw

---

Target:0 (not real diaster)
Text:
@alanhahn @HDumpty39 Daughtery would get destroyed on twitter. His comments are emotionally driven rants with little factual basis

---

Target:0 (not real diaster)
Text:
If it wasn't for the Blood! ????

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# turn dataframe values to dataframe columns
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                    train_df_shuffled["target"].to_numpy(),
                                                    test_size = 0.1, # use 10% of traianing data for validations
                                                    random_state = 42)

In [14]:
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [15]:
# # Reshape data
# train_sentences = train_sentences.reshape(-1, 1)
# train_labels = train_labels.reshape(-1, 1)
# val_labels = val_labels.reshape(-1, 1)
# val_sentences = val_sentences.reshape(-1, 1)
# train_sentences.shape, train_labels.shape, val_labels.shape, val_sentences.shape

In [16]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

    When dealing with a text problem, on of the first things you'll have to do before you can build a model is to convert your text to numbers.

    There are a few ways to do this, namely:
     * Tokenization - direct mapping of token (a token could be a word or a caracter) to number
     * Embedding - create a matric of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

# Text vectorization (tokenization)

In [17]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [18]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorization = TextVectorization(max_tokens=50000, # how many words in the vocabulary(utomatically add <OOV>)
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None, # Create groups of n-words?
                                       output_mode="int", # How to map tokens to numbers
                                       output_sequence_length=None, # how longdo you want your sequences to be?
                                       pad_to_max_tokens=True)

In [19]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [20]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])) # nb of words
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # average for 1 sentence

15

In [21]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be 

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                   output_mode = "int",
                                   output_sequence_length= max_length)

In [22]:
# Fit the text vectorizer  to the training text 
# that convert our words data to numeric format 
text_vectorizer.adapt(train_sentences)

In [23]:
# Create a sample sentences and tokenize it 
sample_sentence = "There's a flood in my street !"
text_vectorizer([sample_sentence])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [24]:
# Choose a random sentence from the training dataset and tokenize it 
random_sentence = random.choice(train_sentences)
print(f"Origina text:\n{random_sentence}\
\n\n Vectorized version:")
text_vectorizer([random_sentence])

Origina text:
LONER DIARIES.

The patterns  on the sand
May have been blown away.
The photos in twos
All choked up in flames.... http://t.co/EKfaZ6wVBz

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    1,    2, 9910,   11,    2, 3445,  133,   24,   59,  468,
         406,    2,  729,    4]], dtype=int64)>

In [25]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating an Embedding using an Embedding Layer

    To make our embedding, we re going to use TensorFlow's embedding layer : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
    The parameters we xare most about for our embedding layer:
    *`input_dim` = the size of our vocabulary
    *`output_dim` = the size of the output embedding vector, for example, a value of 100 could mean each token gets represented by a vector 100 long
    * `input_length` = length of the sequences being passed to the embedding layer

In [26]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length)
embedding

In [27]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
\n\n Embedded version :")
# Embed the random sentence (turn it into danse victors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))

#sample_embed = tf.reshape(sample_embed, (1, 15, 128))
sample_embed

Original text:
Breast milk is the original #superfood but rates worldwide have stalled below 40% contributing to more than 800000 child deaths last year.

 Embedded version :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01862655,  0.00710901, -0.02142434, ...,  0.02005215,
         -0.00837637,  0.00907401],
        [ 0.02223944,  0.03710261,  0.00026308, ...,  0.00520402,
          0.03138144, -0.02601237],
        [ 0.0276008 , -0.01915122,  0.02508228, ...,  0.01264496,
         -0.02279168, -0.01157744],
        ...,
        [-0.01979972, -0.01134163, -0.03134932, ...,  0.01966694,
         -0.03352403, -0.02715345],
        [ 0.04401907, -0.0283611 , -0.02399799, ..., -0.03134312,
         -0.01548762, -0.03114202],
        [-0.045026  ,  0.02001273, -0.00347382, ..., -0.02602495,
         -0.02614348, -0.01164038]]], dtype=float32)>

In [28]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01862655,  0.00710901, -0.02142434, -0.00423529,  0.02773459,
        -0.00956122,  0.01250083, -0.04924045,  0.01739038, -0.00121745,
         0.0420169 , -0.02430445, -0.009904  ,  0.02116014, -0.00643764,
        -0.00316488, -0.04606071, -0.0094581 ,  0.03392006,  0.04117863,
        -0.04298815,  0.03046015, -0.04388681, -0.01994066, -0.02103976,
        -0.00669336, -0.0370665 , -0.01146318,  0.04667498, -0.01260828,
        -0.00749432, -0.02782056,  0.04071749,  0.04053387,  0.02076732,
         0.04117873,  0.03125416,  0.01471877, -0.01256913,  0.03095515,
         0.01260335,  0.03383701, -0.02218709,  0.04204145,  0.01833422,
        -0.00596211, -0.01288508, -0.02231542, -0.02649485,  0.02903146,
         0.02988594,  0.02742939, -0.02166265, -0.04883588,  0.02568698,
        -0.02755943, -0.0305819 ,  0.03548371, -0.04414595,  0.01243754,
        -0.04280796,  0.03719437, -0.04535346, -0.00639183, -0.00674845,
  

## Modelling a text dateset (running a series of experimente)

    Now we ve a got way to turn our text sequences into numbers,
    it's time to start building a series of modelling experiments.

    We'll start with a baseline and moce on from there.

* Model 0: Naive Bayes (baseline) with sklearn : https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1: Feed-forward neural network (danse model)
* Model 2 : LSTM model (RNN)
* model 3 : GRU model (RNN)
* model 4 : Bidirectional-LSTM model (RNN)
* Model 5 : 1D Convolutional Neural Network (CNN)
* Model 6 : TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7 : Same as model 6 with 10% of training data

How are we going to approch all of these ?

Use the standard steps in modelling with tensorflow:

* Create a model 
* Build a model
* Fit a model
* Evaluate model

### Model 0 : Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you're got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayers using the TF-IDF formula to convert our words to numbers

>** note : ** it's common practice to use non-Deep Learning Algorithms as a baseline because of their speed and then later using Deep Learning to to see if you can imporove upon them.

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline ([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text clf : juste classification
])

# Fit the pipleine to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [30]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score}")

Our baseline model achives an accuracy of : 0.7926509186351706


In [31]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]
# for i in range(0,10,1):
#     print(f" {val_sentences[i]} \n prediction : {baseline_preds[i]}")
    

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0], dtype=int64)

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however this will be cumbersome and could easily be fixed with function

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy.
* Precision.
* Recall
* F1-score

For a deep overview of many different evaluation methods, see the sklearn

### 

In [32]:
# Function to evaluate : accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# rajouter
def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                    "precision": model_precision,
                    "recall": model_recall,
                    "f1": model_f1}
    return model_results

In [33]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

# Model 1: A simple dense model

In [34]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [82]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional string
x = text_vectorizer(inputs)# turn the input text into numbers
x = embedding(x) # transform the input numbers in embedding
x = layers.GlobalAveragePooling1D()(x) # Condense the feature vector for each token to on vector
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs
model_1 = tf.keras.Model(inputs,outputs, name="Model_1_dense")

In [83]:
model_1.summary()    

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [84]:
model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [85]:
history_model_1 = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                        experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230110-032805
Epoch 1/5
215/215 [==============================] - 2s 7ms/step - loss: 0.5425 - accuracy: 0.7597 - val_loss: 0.5104 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3283 - accuracy: 0.9032 - val_loss: 0.4608 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2435 - accuracy: 0.9237 - val_loss: 0.4671 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1981 - accuracy: 0.9393 - val_loss: 0.4844 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1678 - accuracy: 0.9483 - val_loss: 0.5066 - val_accuracy: 0.7822


In [86]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.5066 - accuracy: 0.7822


[0.5065644979476929, 0.7821522355079651]

In [40]:
# Make some predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape, model_1_pred_probs[0]

24/24 [==============================] - 0s 2ms/step


((762, 1), array([0.41261524], dtype=float32))

In [41]:
# Look at a single prediction
model_1_pred_probs[:10]

array([[0.41261524],
       [0.8063151 ],
       [0.9976059 ],
       [0.12623218],
       [0.11249132],
       [0.94221914],
       [0.9085771 ],
       [0.9928839 ],
       [0.97254306],
       [0.25391257]], dtype=float32)

In [42]:
model_1_pred_probs.shape

(762, 1)

In [43]:
# Convert model prediction probabilities to label format

model_1_pred_probs = tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred_probs[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [44]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                   y_pred=model_1_pred_probs)
model_1_results

{'accuracy': 78.08398950131233,
 'precision': 0.783783808499639,
 'recall': 0.7808398950131233,
 'f1': 0.7783998521836788}

In [45]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [46]:
# compare witch best between our models
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values())) 

array([False, False, False, False])

## Visualizing learned embeddings


In [47]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [48]:
max_vocab_length

10000

In [49]:
# Model 1 summary
model_1.summary()

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [50]:
# Get the weight matrix of embedding layer
# (thse are the numerical representations of each token in our training data, which have been learned for -5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()
print(embed_weights[0].shape) # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)


In [51]:
embed_weights

[array([[-0.00329006,  0.05695612,  0.02924675, ...,  0.0064363 ,
          0.0173322 , -0.02349018],
        [-0.01499821,  0.0100141 , -0.02299671, ...,  0.02318005,
          0.0018912 ,  0.0057132 ],
        [ 0.00653614, -0.00372134, -0.04173844, ...,  0.00243149,
         -0.02492782, -0.00158993],
        ...,
        [ 0.04223019, -0.02354351, -0.026406  , ...,  0.02043827,
         -0.04619041,  0.01020976],
        [ 0.03928196,  0.05693267, -0.08099077, ...,  0.06834428,
          0.08508166, -0.04633959],
        [ 0.05945378,  0.02234247, -0.02991937, ...,  0.05735027,
          0.07354595, -0.08726752]], dtype=float32)]

Now we've got the embedding matric our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handly tool called projector: https://projector.tensorflow.org/

And TensorFlow also has an incredible guide on word embeddings themselves
https://www.tensorflow.org/text/guide/word_embeddings

In [52]:
# # Create embedding files (we got this from TensorFlow's word embedding documentation)
# import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[0][index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [53]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

Downloading the files above can visualize them usign http://projector.tensorflow.org/ and clicking the "load" button on the left hand
*Ressources : if you'd like to know more about embedding, I'd encourage you to check out :
- Jay Alammer's vusualized word2vec post : https://jalammar.github.io/illustrated-word2vec/
- TensorFlow's Word Embeddings guide: https://www.tensorflow.org/tutorials/text/word_embeddings

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later
* Ressources :> 
If you want an overview of the internale of a recurrent neural network, see the following : 
    - MIT's sequence modelling lecture https://www.youtube.com/watch?v=QvkQ1B3FBqA
    - Chris Olah's intro to LSTMs : https://colah.github.io/posts/2015-08-Understanding-LSTMs/
    - Andrej Karphathy's the unreasonable effectiveness of recurrent neural network : http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - https://www.youtube.com/watch?v=QvkQ1B3FBqA&t=166s

### Model 2 : LSTM 
LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN Typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [54]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units=64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to  returns sequences=True
# print(x.shape)
x = layers.LSTM(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
# print(x.shape)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [55]:
# Get the summary 
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [56]:
# Compile the model
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [57]:
# Fit the model 
history_model2 = model_2.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_2_LSTM")]) 

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230110-031240
Epoch 1/5
215/215 [==============================] - 12s 50ms/step - loss: 0.2671 - accuracy: 0.9126 - val_loss: 0.5199 - val_accuracy: 0.7900
Epoch 2/5
215/215 [==============================] - 11s 50ms/step - loss: 0.1674 - accuracy: 0.9358 - val_loss: 0.6828 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 12s 54ms/step - loss: 0.1366 - accuracy: 0.9475 - val_loss: 0.7988 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 11s 50ms/step - loss: 0.1126 - accuracy: 0.9564 - val_loss: 0.9968 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 14s 63ms/step - loss: 0.0940 - accuracy: 0.9648 - val_loss: 1.1800 - val_accuracy: 0.7756


In [58]:
# Make prediction with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 8ms/step


array([[8.5309021e-02],
       [8.8352269e-01],
       [1.0000000e+00],
       [4.7954641e-02],
       [9.7282245e-05],
       [9.9026299e-01],
       [6.4554250e-01],
       [1.0000000e+00],
       [9.9999976e-01],
       [3.1508851e-01]], dtype=float32)

In [59]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs)) # enlever les dimension en plus
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [60]:
# Calculate model 2 results
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'precision': 0.7820177396132826,
 'recall': 0.7755905511811023,
 'f1': 0.7717056466940521}

In [61]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [62]:
np.array(list(model_1_results.values())) > np.array(list(model_2_results.values()))  

array([ True,  True,  True,  True])

### Model 3 : GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [196]:
# Build an RNN GRU
inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GRU(3)(x)
# x = tf.keras.layers.GRU(units=64, return_sequences=True)(x) 
# x = tf.keras.layers.LSTM(42, return_sequences=True)(x)
# x = tf.keras.layers.GRU(units=92, return_sequences=True)(x)

# # Apply global average pooling
# x = tf.keras.layers.GlobalAveragePooling1D()(x)

# x = tf.keras.layers.Dense(64, activation="relu")(x)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [197]:
# # Version 2
# # Build an RNN GRU
# inputs = tf.keras.layers.Input(shape=(1), dtype=tf.string)
# x = text_vectorizer(inputs)
# x = embedding(x)
# # x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.GRU(units=64, return_sequences=True)(x) 
# x = tf.keras.layers.LSTM(42, return_sequences=True)(x)
# x = tf.keras.layers.GRU(units=92, return_sequences=True)(x)

# # Apply global average pooling
# x = tf.keras.layers.GlobalAveragePooling1D()(x)

# x = tf.keras.layers.Dense(64, activation="relu")(x)
# outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
# model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [198]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 gru_44 (GRU)                (None, 3)                 1197      
                                                                 
 dense_51 (Dense)            (None, 1)                 4         
                                                                 
Total params: 1,281,201
Trainable params: 1,281,201
Non-trainable params: 0
_____________________________________________

In [199]:
# compile the model
model_3.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [200]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                        experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230110-041511
Epoch 1/5
215/215 [==============================] - 4s 11ms/step - loss: 0.3714 - accuracy: 0.8643 - val_loss: 0.4914 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1508 - accuracy: 0.9661 - val_loss: 0.5362 - val_accuracy: 0.7585
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0997 - accuracy: 0.9772 - val_loss: 0.6010 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0762 - accuracy: 0.9799 - val_loss: 0.6601 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0634 - accuracy: 0.9807 - val_loss: 0.7108 - val_accuracy: 0.7559


In [201]:
# Make som prediction with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)

24/24 [==============================] - 0s 3ms/step


In [202]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))

In [203]:
model_3_results = calculate_results(val_labels, model_3_preds)
model_3_results

{'accuracy': 75.59055118110236,
 'precision': 0.756358217141213,
 'recall': 0.7559055118110236,
 'f1': 0.7542126125197939}

In [205]:
# comparing model 1 with model 2
np.array(list(model_1_results.values())) > np.array(list(model_3_results.values()))  

array([ True,  True,  True,  True])

# Mode 4 :  Bidirectional RNN



In [241]:
# create the model
inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation="relu"))(x)
# x = tf.keras.layers.AveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional_layer")

In [242]:
model_4.summary()

Model: "model_4_Bidirectional_layer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_63 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 bidirectional_20 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dense_73 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable param

In [243]:
model_4_predict_prob = tf.squeeze(model_4.predict(val_sentences))
model_4_preds = tf.round(model_4_predict_prob)

24/24 [==============================] - 1s 13ms/step


In [240]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 46.06299212598425,
 'precision': 0.5973688719373655,
 'recall': 0.46062992125984253,
 'f1': 0.29942509848518084}

# Model 4 : Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence), however, bidirectional RNN goes from right to left as well as left to right.



In [254]:
# Build a bidirectional RNN in TensorFlow 
# create the model
inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(x)
print(x.shape) # nb_units * 2 comme c'est bidirectional

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional_layer")

(None, 128)


In [255]:
 # Get a summary 
model_4.summary()

Model: "model_4_Bidirectional_layer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_68 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 bidirectional_29 (Bidirecti  (None, 15, 128)          98816     
 onal)                                                           
                                                                 
 bidirectional_30 (Bidirecti  (None, 128)              74496     
 onal)                                                           
                                       

In [256]:
# Compile the model
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [268]:
history_model_4 = model_4.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels),
                             callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name="Bidirectionalmod")])

Saving TensorBoard log files to: model_logs/Bidirectionalmod/20230110-055225
Epoch 1/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0272 - accuracy: 0.9858 - val_loss: 1.9373 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0290 - accuracy: 0.9841 - val_loss: 1.8545 - val_accuracy: 0.7598
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0285 - accuracy: 0.9847 - val_loss: 1.6445 - val_accuracy: 0.7546
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0292 - accuracy: 0.9851 - val_loss: 1.6864 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0279 - accuracy: 0.9851 - val_loss: 1.9744 - val_accuracy: 0.7572


In [269]:
# Make prediction with our bidirectional model
model_4_predict_prob = tf.squeeze(model_4.predict(val_sentences))
# Convert pred probs to labels 
model_4_preds = tf.round(model_4_predict_prob)

24/24 [==============================] - 1s 9ms/step


In [270]:
# Convert pred probs to labels 
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 75.7217847769029,
 'precision': 0.7576149234046069,
 'recall': 0.7572178477690289,
 'f1': 0.7555947144839635}